In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

data = []
labels = []
i = 0

In [ ]:
for root, dirs, files in os.walk('/Users/akshatgupta/Desktop/IIITD/SEM_4/SML/Assignment1/trainingSet'):
  for j in files:
    if j.endswith('jpg'):
      image = Image.open(os.path.join(root,j))
      image = np.array(image)[:,:]
      image = np.reshape(image, (image.shape[0] * image.shape[1]))
      data.append(image)
      labels.append(i)
  i+=1

In [ ]:
data = np.array(data)
labels = np.array(labels)
row = np.shape(data)[0]
height = np.shape(labels)[0]

training_data, testing_data = np.split(data, [int(row*4/5)])

training_label, testing_label = np.split(labels, [int(4/5*height)])

In [ ]:
def pca(data):
  means = np.mean(data, axis=0)
  
  centered_data = data - means

  cov_centered_data = np.cov(centered_data, rowvar=False)
  
  eigenvalues, eigenvectors = np.linalg.eig(cov_centered_data)

  eigenvectors = eigenvectors[:,np.argsort(eigenvalues)[::-1]]
  eigenvalues = np.sort(eigenvalues)[::-1]

  components = np.dot(centered_data, eigenvectors)

  return components

In [ ]:
def kNN(training_data,k,testing_data):
  kNN_testing_labels = np.zeros(len(testing_data))
  
  for i in range(len(testing_data)):
    distances = []
    for j in range(len(training_data)):
      diff = np.abs((testing_data[i] - training_data[j])**2)
      distance = np.sum(diff)
      distances.append(distance)
    
  padosi = training_label[np.argsort(distances,axis=1)[:,:k]]

  for i,p in enumerate(padosi):
    kNN_testing_labels[i] = np.bincount(p).argmax()

  return kNN_testing_labels

In [ ]:
pca_1 = pca(training_data)
pca_2 = pca(testing_data)

pca_1_5 = pca_1[:,:5]
pca_2_5 = pca_2[:,:5]

pca_1_25 = pca_1[:,:25]
pca_2_25 = pca_2[:,:25]

pca_1_125 = pca_1[:,:125]
pca_2_125 = pca_2[:,:125]

In [ ]:
kNN_1 = kNN(testing_data,5,training_data)
kNN_1_5 = kNN(pca_1_5,5,pca_2_5)
kNN_1_25 = kNN(pca_1_25,25,pca_2_25)
kNN_1_125 = kNN(pca_1_125,125,pca_2_125)

In [ ]:
#Accuracies

A_kNN_1 = sum([1 for x,y in zip(kNN_1,training_label) if x==y])
print("Accuracy of kNN on training_data: ", 100*A_kNN_1/(len(kNN_1)))

A_kNN_1_5 = sum([1 for x,y in zip(kNN_1,training_label) if x==y])
print("Accuracy of kNN on pca components for k=5: ", 100*A_kNN_1_5/(len(kNN_1_5)))

A_kNN_1_25 = sum([1 for x,y in zip(kNN_1,training_label) if x==y])
print("Accuracy of kNN on pca components for k=25: ", 100*A_kNN_1_25/(len(kNN_1_25)))

A_kNN_1_125 = sum([1 for x,y in zip(kNN_1,training_label) if x==y])
print("Accuracy of kNN on pca components for k=125: ", 100*A_kNN_1_125/(len(kNN_1_125)))


In [ ]:
means = np.mean(training_data, axis=0)
  
centered_data = training_data - means
cov_centered_data = np.cov(centered_data, rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov_centered_data)
eigenvalues = np.sort(eigenvalues)[::-1]

esum = np.sum(eigenvalues)
values1 = eigenvalues / esum
values = np.cumsum(values1)

plt.plot(range(1,len(values)+1),values)
plt.xlabel("Principal Components")
plt.ylabel("Explained Variance Ratio")
plt.show()

In [ ]:
print("Number of PCs required to cover 80% of variance: ",np.argmax(values >= 0.8) + 1)